<a href="https://colab.research.google.com/github/aksj98/EmpiricallyEmergent/blob/main/BERT_Probing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tutorial for probing BERT on the IMDB movie review dataset 

In [ ]:
#@title Install required libraries
!pip install transformers
!pip install torch
!pip install datasets

In [2]:
#@title Imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel, AdamW, Trainer, TrainingArguments
from datasets import load_dataset